# **FIND CLAPS IN VIDEO**

In [1]:
import os
from clap_detection_methods import process_videos_in_directory, load_accelerometer_data, plot_accelerometer_data_interval, normalize_data, find_peaks_in_interval

In [ ]:
directory_path = os.path.join(os.path.dirname(os.getcwd()), "Data\\videos")
audio_output_path = os.path.join(os.path.dirname(os.getcwd()), "Data\\audio")
clap_results = process_videos_in_directory(directory_path, num_segments= 4, audio_output_dir=audio_output_path)

In [ ]:
import pandas as pd

df_clap_results = pd.DataFrame(clap_results)
df_clap_results

# **FIND PEAKS IN ACCELERATION DATA**

In [ ]:
acc_data_path = "C:/Users/niki/Desktop/Step-counter/Data/STM2D48231610 (2024-11-28)RAW.csv"
sampling_frequency = 256
cleaned_data, time_seconds = load_accelerometer_data(acc_data_path, sampling_frequency)

plot_accelerometer_data_interval(cleaned_data, time_seconds, title_suffix="Full Duration", plot_each_axis=False)

In [ ]:
plot_accelerometer_data_interval(cleaned_data, time_seconds, start_time=600, end_time=None, title_suffix="Full Duration", plot_each_axis=False)

In [ ]:
plot_accelerometer_data_interval(cleaned_data, time_seconds, start_time=850, end_time=900, title_suffix="Full Duration")

In [7]:
start_time = 855
end_time = 900

In [ ]:
# Find the peaks in the interval
normalized_data = normalize_data(cleaned_data)
first_half_time, second_half_time, index_first_clap, index_last_clap = find_peaks_in_interval(normalized_data, time_seconds, start_time, end_time)

# Print the results
print("Peaks in the interval:")
print(f"First Half Peak - Time: {first_half_time:.2f}")
print(f"Second Half Peak - Time: {second_half_time:.2f}")
print(f"First Half Peak - Index: {index_first_clap}")
print(f"Second Half Peak - Index: {index_last_clap}")
duration = second_half_time - first_half_time
print(f"Total Duration: {duration:.3f}s")

# **SYNC DATA**
